# Загрузка библиотек. Подготовка данных

In [1]:
import csv
import os
from itertools import combinations
import glob
import pandas as pd

# Определяем гапаксы для каждой коллекции и их соотношение к общему числу лемм в каждом сборнике

In [4]:
# Папка с cleaned-файлами
lemmas_dir = os.path.join("..", "data", "lemmas")
files = glob.glob(os.path.join(lemmas_dir, "cleaned_*.csv"))

# Папка для сохранения результата
output_dir = os.path.join("..", "output", "stats_tables")
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "unique_lemma_stats.csv")

# Собираем все данные
all_data = []

for file in files:
    try:
        # Читаем файл без заголовка, задаём имя столбца "lemma"
        df = pd.read_csv(file, header=None, names=["lemma"], encoding="utf-8")

        # Получаем название сборника из имени файла
        filename = os.path.basename(file).replace("cleaned_", "")

        # Добавляем столбец с названием сборника
        df["source"] = filename

        # Убираем дубликаты по лемме и сборнику
        df = df.drop_duplicates(subset=["lemma", "source"])

        all_data.append(df)
    except Exception as e:
        print("f Ошибка при обработке {file}: {e}")

# Объединяем все данные
df_all = pd.concat(all_data, ignore_index=True)

# Подсчёт количества сборников, в которых встречается каждая лемма
df_counts = df_all.groupby("lemma")["source"].nunique().reset_index()
df_counts.columns = ["lemma", "collection_count"]

# Добавляем эту информацию в основной DataFrame
df_all = df_all.merge(df_counts, on="lemma", how="left")

# Отмечаем, какие леммы уникальны
df_all["is_unique"] = df_all["collection_count"] == 1

# Группировка по сборнику и подсчёт уникальных/неуникальных лемм
df_stats = df_all.groupby("source")["is_unique"].value_counts().unstack().fillna(0)
df_stats.columns = ["не уникальные леммы", "уникальные леммы"]

# Добавляем общее количество и процент уникальных
df_stats["всего лемм"] = (
    df_stats["не уникальные леммы"] + df_stats["уникальные леммы"]
)
df_stats["% уникальных лемм"] = (
    df_stats["уникальные леммы"] / df_stats["всего лемм"] * 100
).round(2)

# Сохраняем результат
df_stats.to_csv(output_path, sep=";", encoding="utf-8")

print(f"Статистика сохранена: {output_path}")


Статистика сохранена: ../output/stats_tables/unique_lemma_stats.csv


# Создаем таблицу соотношения уникальных лемм к неуникальным и сохраняем в csv

In [5]:
df_stats

,не уникальные леммы,уникальные леммы,всего лемм,% уникальных лемм
source,,,,
alanus_de_insula_dist_summa.predicted_лем.csv,941,484,1425,33.96
anonymous_angelus.predicted_лем.csv,625,325,950,34.21
guido_ebroicensis_summa.predicted_лем.csv,70,5,75,6.67
mauritius_hibernicus_distinctiones.predicted_лем.csv,641,488,1129,43.22
nicolas_de_biard-distinctiones.predicted_лем.csv,555,311,866,35.91
nicolas_de_gorran_distinctiones.predicted_лем.csv,312,58,370,15.68
petrus_cantor_distinctiones.predicted_лем.csv,715,531,1246,42.62


In [6]:
df_stats.to_csv('../output/stats_tables/стастистика_по_леммам.csv')

# Создаем и сохраняем в csv таблицу типов лемм, которая покажет, является ли лемма уникальной; если нет, то в каком количестве сборников она встречается.

In [7]:
df_all

,lemma,source,collection_count,is_unique
0,lemma,anonymous_angelus.predicted_лем.csv,7.0,False
1,angelus,anonymous_angelus.predicted_лем.csv,6.0,False
2,anima,anonymous_angelus.predicted_лем.csv,6.0,False
3,Abyssus,anonymous_angelus.predicted_лем.csv,4.0,False
4,Abies,anonymous_angelus.predicted_лем.csv,2.0,False
...,...,...,...,...
6056,unitas,mauritius_hibernicus_distinctiones.predicted_л...,3.0,False
6057,utilitas,mauritius_hibernicus_distinctiones.predicted_л...,2.0,False
6058,z,mauritius_hibernicus_distinctiones.predicted_л...,1.0,True
6059,zelus,mauritius_hibernicus_distinctiones.predicted_л...,6.0,False


In [8]:
df_all.to_csv('../output/stats_tables/стастистика_по_леммам2.csv')